<a href="https://colab.research.google.com/github/akitim/SkillFactory_current/blob/main/MATH%26ML_8_%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D1%8C%D0%B5%D0%B2_%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 3.4
1/1 point (graded)

Дана часть датасета о домах в штате Айова, состоящая из 15 наблюдений:

Найдите уникальных кандидатов на пороговое значение  для фактора *Lot Frontage*.

In [2]:
import numpy as np
import pandas as pd

# path = '/content/AmesHousing.zip'
path = 'https://raw.githubusercontent.com/akitim/datasets/main/AmesHousing.zip'
ames_data = pd.read_csv(path)
ames_data = ames_data[(ames_data['Lot Frontage'] <=150) & (ames_data['Lot Area'] <=20000)]
ames_sample = ames_data[['Lot Frontage', 'Lot Area', 'SalePrice']].sample(
    n=15, #количество объектов
    random_state=42, #генератор случайных чисел
    ignore_index=True #сбрасываем счетчик индекса
)
ames_sample

,Lot Frontage,Lot Area,SalePrice
0,60.0,11040,108000
1,96.0,10628,193000
2,68.0,9717,142125
3,50.0,9060,150909
4,92.0,9215,204500
5,80.0,10386,305900
6,60.0,7350,139000
7,72.0,8640,229800
8,56.0,6956,97900
9,70.0,12886,175000


In [3]:
LF_sorted = ames_sample['Lot Frontage'].sort_values().drop_duplicates()
# LF_sorted
candidates = LF_sorted.rolling(2).mean().dropna()
candidates

8      53.0
0      58.0
14     61.0
2      65.0
9      69.0
7      71.0
5      76.0
4      86.0
1      94.0
12    101.0
Name: Lot Frontage, dtype: float64

Задание 3.5
1 point possible (graded)

Для части датасета о домах в Айове из предыдущего задания найдите уникальных кандидатов на пороговое значение  для фактора *Lot Area*.

In [4]:
ames_sample['Lot Area'].sort_values().drop_duplicates().\
    rolling(2).mean().dropna()

8      5508.0
6      7153.0
14     7425.0
13     7949.0
12     8405.5
7      8526.5
3      8850.0
4      9137.5
2      9466.0
5     10051.5
1     10507.0
0     10834.0
11    11365.0
9     12288.0
Name: Lot Area, dtype: float64

In [12]:
def find_candidates_for_thresholds(x, y):
    x_roll_mean = x.sort_values().drop_duplicates().rolling(2).mean().dropna()
    y_roll_mean = y[x_roll_mean.index].diff()
    return x_roll_mean[y_roll_mean != 0].values

def squared_error(y):
    y_pred = np.mean(y)
    return (y - y_pred) @ (y - y_pred) / len(y)

def split(X, y, split_params):
    X_ = X_.copy()
    if type(X_) in [pd.DataFrame, pd.Series]:
        X_ = X_.values
    j, t = split_params
    predicat = X[:, j] <= t
    X_left, y_left, X_right, y_right = \
        X_[predicat, j], y[predicat], X_[~predicat, j], y[~predicat]
    return X_left, y_left, X_right, y_right

def calculate_weighted_impurity(X, y, split_params, crit):
    X_left, y_left, X_right, y_right = split(X, y, split_params)
    N, N_left, N_right = y.size, y_left.size, y_right.size
    return N_left / N * crit(y_left) + N_right / N * crit(y_right)

def best_split(X, y, crit):
    M = X.shape[1]
    min_wt_imp = np.inf
    opt_split_params = None
    for j in range(M):
        thrs = find_candidates_for_thresholds(X[:, j], y)
        for t in thrs:
            split_params = j, t
            wt_imp = calculate_weighted_impurity(X, y, split_params, crit)
            if wt_imp < min_wt_imp:
                min_wt_imp = wt_imp
                opt_split_params = split_params
    return opt_split_params


In [13]:
X = ames_sample[['Lot Frontage', 'Lot Area']]
y = ames_sample['SalePrice']

best_split(X, y, squared_error)

InvalidIndexError: ignored